https://github.com/crahan/ipyfilechooser

In [1]:
# !pip install ipyfilechooser

In [2]:
import os
import geogee
import ipywidgets as widgets
from ipyleaflet import WidgetControl
from ipyfilechooser import FileChooser

C:\Users\Sarath_Kin\miniconda3\envs\demo\lib\site-packages\google\api_core\_python_version_support.py:237: FutureWarning: You are using a non-supported Python version (3.8.20). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)


An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


## Getting started with ipyfilechooser

In [3]:
data_dir = os.path.abspath('./data')

fc = FileChooser(data_dir)
fc.use_dir_icons = True
fc.filter_pattern = ['*.shp', '*.geojson']

display(fc)

FileChooser(path='L:\Coding\Github\geogee\examples\data', filename='', title='', show_hidden=False, select_des…

In [4]:
print(fc.selected_path)

None


In [5]:
print(fc.selected_filename)

None


In [6]:
print(fc.selected)

None


In [7]:
fc.reset()

## Creating a toolbar button

In [8]:
widget_width = "250px"
padding = "0px 0px 0px 5px"  # upper, right, bottom, left

toolbar_button = widgets.ToggleButton(
    value=False,
    tooltip="Toolbar",
    icon="wrench",
    layout=widgets.Layout(width="28px", height="28px", padding=padding),
)

close_button = widgets.ToggleButton(
    value=False,
    tooltip="Close the tool",
    icon="times",
    button_style="primary",
    layout=widgets.Layout(height="28px", width="28px", padding=padding),
)

In [9]:
toolbar = widgets.HBox([toolbar_button])
toolbar

## Adding toolbar event

In [10]:
def toolbar_click(change):
    if change["new"]:
        toolbar.children = [toolbar_button, close_button]
    else:
        toolbar.children = [toolbar_button]
        
toolbar_button.observe(toolbar_click, "value")

In [11]:
def close_click(change):
    if change["new"]:
        toolbar_button.close()
        close_button.close()
        toolbar.close()
        
close_button.observe(close_click, "value")
toolbar

## Adding toolbar grid

In [12]:
rows = 2
cols = 2
grid = widgets.GridspecLayout(rows, cols, grid_gap="0px", layout=widgets.Layout(width="62px"))

icons: https://fontawesome.com/v4.7.0/icons/

In [13]:
icons = ["folder-open", "map", "info", "question"]

for i in range(rows):
    for j in range(cols):
        grid[i, j] = widgets.Button(description="", button_style="primary", icon=icons[i*rows+j], 
                                    layout=widgets.Layout(width="28px", padding="0px"))
grid

GridspecLayout(children=(Button(button_style='primary', icon='folder-open', layout=Layout(grid_area='widget001…

In [14]:
toolbar = widgets.VBox([toolbar_button])

In [15]:
def toolbar_click(change):
    if change["new"]:
        toolbar.children = [widgets.HBox([close_button, toolbar_button]), grid]
    else:
        toolbar.children = [toolbar_button]
        
toolbar_button.observe(toolbar_click, "value")
toolbar

## Adding toolbar to ipyleaflet

In [16]:
toolbar_ctrl = WidgetControl(widget=toolbar, position="topright")

In [24]:
m = geogee.Map()
m.add_control(toolbar_ctrl)
m

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [25]:
output = widgets.Output()
output_ctrl = WidgetControl(widget=output, position="topright")

In [26]:
buttons = widgets.ToggleButtons(
    value=None,
    options=["Apply", "Reset", "Close"],
    tooltips=["Apply", "Reset", "Close"],
    button_style="primary",
)
buttons.style.button_width = "80px"

In [27]:
filechooser_widget = widgets.VBox([fc, buttons])

In [28]:
def button_click(change):
    if change["new"] == "Apply" and fc.selected is not None:
        if fc.selected.endswith(".shp"):
            m.add_shapefile(fc.selected, layer_name="Shapefile")
        elif fc.selected.endswith(".geojson"):
            m.add_geojson(fc.selected, layer_name="GeoJSON")
    elif change["new"] == "Reset":
        fc.reset()
    elif change["new"] == "Close":
        fc.reset()
        m.remove_control(output_ctrl)
buttons.observe(button_click, "value")        

In [29]:
def tool_click(b):    
    with output:
        output.clear_output()
        if b.icon == "folder-open":
            display(filechooser_widget)
            m.add_control(output_ctrl)

In [30]:
for i in range(rows):
    for j in range(cols):
        tool = grid[i, j]
        tool.on_click(tool_click)

![](https://i.imgur.com/q4ub8uC.gif)